# 动态规划算法
这部分算法包含两部分：策略迭代算法和价值迭代算法。

## 策略迭代算法
### 第一步：状态价值评估
从状态价值最优Bellman方程出发：

$V^{\pi}(s)=\sum_{a \in A}\pi(a|s)\left(r(s, a)+\gamma \sum_{s' \in S}P(s'|s, a)V^{\pi}(s')\right)$

可以将策略价值函数改写为一个递归的形式：

$V^{k+1}(s)=\sum_{a \in A}\pi(a|s)\left(r(s, a)+\gamma \sum_{s' \in S}P(s'|s, a)V^{k}(s')\right)$

其中$k$表示经过$k$次迭代的策略价值函数。

### 第二步：策略提升

策略提升定理：$Q^{\pi}(s, \pi' (s))\geq V^{\pi}(s)$，可以说明策略$\pi'$更优。

用贪心的方法确定新的策略：
$\pi' (s) = \text{argmax}_a Q^{\pi}(s, a) = \text{argmax}_a \left(r(s, a)+\gamma \sum_{s' \in S}P(s'|s, a)V^{k}(s)\right) $

In [15]:
import numpy as np

# 构建悬崖行走的环境
class CliffWalkingEnv:
    def __init__(self, nrow=4, ncolumn=12):
        # 总共具有的位置为nrow、ncolumn，每个位置存储动作数组，每个动作存储可能的到达的状态，状态包含四个信息（概率、新状态、奖励、done）
        self.nrow = nrow        #定义行
        self.ncolumn = ncolumn  #定义列
        self.P = self.createP()
    
    def createP(self):
        action = [[0, -1], [1, 0], [0, 1], [-1, 0]]
        P = np.zeros([self.nrow*self.ncolumn, len(action), 4])      #这样设计不合理，因为状态因该是一个整数
        for i in range(self.nrow):
            for j in range(self.ncolumn):
                for l, _ in enumerate(action):
                    done = 0
                    reward = 0
                    new_row = min(self.nrow-1, max(0, i+action[l][0]))
                    new_column = min(self.ncolumn-1, max(0, j+action[l][1]))
                    #设置终止位置(不包含奖励终止)
                    if i == self.nrow-1 and j>0 and j!=self.ncolumn-1:
                        done = 1
                        reward = 0
                        P[i*self.ncolumn + j, l] = [1, int(i*self.ncolumn + j), reward, done]
                        continue
                    #设置奖励
                    if new_row == self.nrow-1 and new_column == self.ncolumn-1:
                        reward = 1
                        done = 1
                    state = new_row*self.ncolumn + new_column
                    P[i*self.ncolumn + j, l] = [1, int(state), reward, done]
        return P

In [2]:
#策略价值函数迭代计算
class PolicyIteration:
    def __init__(self, env, gamma, theta):
        self.env = env
        self.gamma = gamma      #折扣比例
        self.theta = theta      #收敛值
        self.V = np.zeros(env.nrow * env.ncolumn)
        self.Pi = np.zeros([env.nrow * env.ncolumn, 4]) + 0.25

    #策略评估
    def Policy_evaluation(self):
        while True:
            #一次迭代过程
            new_V = np.zeros(self.env.nrow * self.env.ncolumn)
            for state in range(self.env.nrow * self.env.ncolumn):
                action = [[0, -1], [1, 0], [0, 1], [-1, 0]]
                for num_act, act in enumerate(action):
                    #这部分因该包含一个针对所有可能情况的循环，这里每个动作只有一个确定的结果，因此没有循环
                    p, new_state, reward, done = self.env.P[state, num_act]
                    new_V[state] += self.Pi[state, num_act] * (reward + self.gamma*p*self.V[int(new_state)])
            # 计算两次迭代的收敛差距
            if np.max(np.abs(self.V-new_V))<self.theta:
                break
            #更新状态价值函数
            self.V = np.copy(new_V)
        return new_V

    #策略提升
    def Policy_improvement(self):
        for state in range(self.env.nrow * self.env.ncolumn):
            # 得到每个新状态的状态价值函数
            act_state_value = np.zeros(4)
            for act in range(4):
                _, new_stat, _, _ = self.env.P[state, act]
                act_state_value[act] = self.V[int(new_stat)]
            # 将动作的状态价值函数归一化，作为新的策略
            if np.sum(act_state_value) > 0.01:
                act_state_value = act_state_value/np.sum(act_state_value)
                self.Pi[state] = act_state_value
        return self.Pi

    #策略迭代
    def Policy_iteration(self):
        while True:
            self.Policy_evaluation()
            old_Pi = np.copy(self.Pi)
            self.Policy_improvement()
            if np.max(np.abs(old_Pi - self.Pi))==0 : break


In [14]:
env = CliffWalkingEnv()
agent = PolicyIteration(env, 0.9, 0.001)
agent.Policy_iteration()
print(agent.V)

[0.05094753 0.06768305 0.09613392 0.1374329  0.19617365 0.27878639
 0.39332695 0.54908521 0.75470257 1.0128891  1.30616783 1.56135029
 0.05274633 0.07163258 0.10258081 0.14774014 0.2128749  0.30627885
 0.43947875 0.62795746 0.89169909 1.25242685 1.71972427 2.22846918
 0.05390552 0.07835528 0.11320432 0.16400502 0.23791368 0.34538327
 0.5016966  0.7298125  1.06448268 1.5598615  2.30204277 3.43022506
 0.04226094 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         4.53290339]


# 价值迭代算法
以贪心的方式，直接对价值函数进行迭代。

从状态价值最优Bellman方程出发：

$V^{\pi}(s)=\text{max}_{a \in A}\left(r(s, a)+\gamma \sum_{s' \in S}P(s'|s, a)V^{\pi}(s')\right)$

进而改写成递归的形式：

$V^{k+1}(s)=\text{max}_{a \in A}\left(r(s, a)+\gamma \sum_{s' \in S}P(s'|s, a)V^{k}(s')\right)$

In [58]:
# 价值迭代函数
class ValueIteration:
    def __init__(self, env, gamma):
        self.env = env
        self.gamma = gamma
        self.V = np.zeros(env.nrow * env.ncolumn)
        self.Pi = self.Pi = np.zeros([env.nrow * env.ncolumn, 4])

    def value_iteration(self):
        old_V = np.zeros(env.nrow * env.ncolumn)
        while True:
            # 一次迭代的过程
            for state in range(self.env.nrow * self.env.ncolumn):
                action = [[0, -1], [1, 0], [0, 1], [-1, 0]]
                new_V = np.zeros(len(action))
                for num_act, act in enumerate(action):
                    p, new_state, reward, _ = self.env.P[state, num_act]
                    new_V[num_act] = reward+self.gamma*p*self.V[int(new_state)]
                self.V[state] = np.max(new_V)
            # 判断是否收敛
            if np.max(self.V - old_V) == 0: break
            else: old_V = np.copy(self.V)
        return self.V

    #利用状态价值函数，产生贪婪策略
    def get_policy(self):
        self.value_iteration()
        for state in range(self.env.nrow * self.env.ncolumn):
            state_value = np.zeros(4)
            for num_act in range(4):
                _, new_state, _, _ = self.env.P[state, num_act]
                state_value[num_act] = self.V[int(new_state)]
            self.Pi[state, np.argmax(state_value)] = 1  #这里只返回靠前的下标，最终的策略只是最优策略的一种
        return self.Pi

In [59]:
env = CliffWalkingEnv()
agent = ValueIteration(env, 0.9)
agent.get_policy()

print(agent.Pi)

[[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]
